In [1]:
import json

with open('name_table.json') as f:
    config = json.load(f)

table_name = config['table_name']

In [2]:
import psycopg2

def find_columns(table_name, conn):
    # Ключевые слова для поиска времени и типа события
    event_time_keywords = ['event', 'time', 'timestamp', 'date']
    event_type_keywords = ['event', 'type', 'category', 'action']

    def column_score(column_name, data_type, keywords, target_types):
        score = 0
        column_name_lower = column_name.lower()

        # Увеличиваем вес для точных ключевых слов
        for keyword in keywords:
            if keyword in column_name_lower:
                score += 2
            if column_name_lower.startswith(keyword) or column_name_lower.endswith(keyword):
                score += 1

        # Увеличиваем вес, если тип данных совпадает с целевым
        if data_type in target_types:
            score += 3
        return score

    # Подключение и выполнение запроса для получения столбцов таблицы с их типами данных
    with conn.cursor() as cur:
        cur.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = %s
        """, (table_name,))
        
        columns = cur.fetchall()

    # Поиск столбца для времени события
    event_time_column = max(
        columns, 
        key=lambda col: column_score(col[0], col[1], event_time_keywords, ['Timestamp', 'time'])
    )

    # Поиск столбца для типа события
    event_type_column = max(
        columns, 
        key=lambda col: column_score(col[0], col[1], event_type_keywords, ['text'])
    )

    return event_time_column[0], event_type_column[0]

# Пример подключения к базе данных
conn = psycopg2.connect(
    user="postgres",
    password="1",
    host="localhost",
    port="5435",
    database="postgres"
)

# Название таблицы
table_name = "anti_frod"

# Поиск столбцов
event_time_column, event_type_column = find_columns(table_name, conn)
print(f"Event Time Column: {event_time_column}")
print(f"Event Type Column: {event_type_column}")

# Закрытие соединения
conn.close()


Event Time Column: event_time
Event Type Column: event_type


In [3]:
import psycopg2
import pandas as pd

# Настройка соединения с базой данных PostgreSQL
conn = psycopg2.connect(
    user="postgres",
    password="1",
    host="localhost",
    port="5435",
    database="postgres"
)

# Название таблицы
table_name = "anti_frod"

# Используем pandas для загрузки данных в DataFrame
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# Закрываем соединение
conn.close()


C:\Users\idukh\AppData\Local\Temp\ipykernel_26660\1802237097.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)


In [4]:
df

,atm_acquiring_country,atm_acquiring_icc,atm_branch,atm_card_entry_mode,atm_cardless,atm_cash_in,atm_cash_in_acc,atm_cash_in_pay,atm_city,atm_class,...,user_doc_number,user_doc_type,user_first_name,user_id,user_last_name,user_name,user_patronymic,user_segment,user_type,usr_ip_cur_add
0,UK,ICC_9910,Branch_1,CONTACTLESS,True,True,True,True,Manchester,VIP,...,DOC_1437,Driver License,FirstName_6,USER_5745,LastName_28,UserName_11,Patronymic_54,Gold,Admin,10.0.0.1
1,Russia,ICC_8912,Branch_5,CHIP,False,False,True,True,Moscow,Standard,...,DOC_3699,ID Card,FirstName_37,USER_1090,LastName_74,UserName_87,Patronymic_67,Gold,Admin,192.168.1.1
2,France,ICC_6210,Branch_30,MAGNETIC_STRIPE,True,False,True,False,Lyon,VIP,...,DOC_7611,ID Card,FirstName_50,USER_5791,LastName_46,UserName_50,Patronymic_68,Bronze,Guest,10.0.0.1
3,France,ICC_7086,Branch_47,CHIP,True,True,True,False,Marseille,Premium,...,DOC_6834,Driver License,FirstName_56,USER_3766,LastName_1,UserName_48,Patronymic_52,Gold,Admin,192.168.1.1
4,Russia,ICC_2583,Branch_19,CHIP,True,False,False,True,Saint Petersburg,VIP,...,DOC_8760,Driver License,FirstName_1,USER_9514,LastName_86,UserName_6,Patronymic_63,Gold,User,172.16.0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,USA,ICC_2058,Branch_33,CONTACTLESS,True,False,True,False,Chicago,Premium,...,DOC_1547,ID Card,FirstName_32,USER_5363,LastName_82,UserName_80,Patronymic_59,Gold,Admin,172.16.0.1
2996,UK,ICC_2398,Branch_2,MAGNETIC_STRIPE,False,True,False,False,Birmingham,Standard,...,DOC_7672,Driver License,FirstName_20,USER_7703,LastName_7,UserName_90,Patronymic_54,Silver,User,192.168.1.1
2997,Germany,ICC_6302,Branch_38,CONTACTLESS,True,True,True,False,Munich,Standard,...,DOC_4195,ID Card,FirstName_97,USER_2438,LastName_34,UserName_60,Patronymic_4,Gold,Guest,10.0.0.1
2998,USA,ICC_7852,Branch_14,CONTACTLESS,True,False,True,True,New York,Standard,...,DOC_4949,Passport,FirstName_60,USER_4655,LastName_98,UserName_90,Patronymic_51,Gold,Guest,10.0.0.1


Time_1

In [5]:
final_query_time_1 = f"SELECT \"{event_time_column}\", \"{event_type_column}\" \n   FROM {table_name} "

In [6]:
Time_1 = (
"table_name: Time_1",
"main_dttm_col: event_time",
"description: null",
"default_endpoint: null",
"offset: 0",
"cache_timeout: null",
"catalog: postgres",
"schema: public",
f"sql: {(final_query_time_1)} ",
"params: null",
"template_params: null",
"filter_select_enabled: true",
"fetch_values_predicate: null",
"extra: null",
"normalize_columns: false",
"always_filter_main_dttm: false",
"uuid: 619ac62e-8715-4084-823e-0cb2e520755e",
"metrics:",
"- metric_name: count",
"  verbose_name: COUNT(*)",
"  metric_type: count",
"  expression: COUNT(*)",
"  description: null",
"  d3format: null",
"  currency: null",
"  extra: null",
"  warning_text: null",
"columns:",
"- column_name: event_time",
"  verbose_name: null",
"  is_dttm: true",
"  is_active: true",
"  type: DATETIME",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"- column_name: event_type",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: STRING",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"version: 1.0.0",
"database_uuid: 2f651e35-38e7-4b9b-a47e-87996c769118"
)

Time_2

In [7]:
import sqlite3

columns =  df.columns.tolist()

count_parts = [f"COUNT('{column}')" for column in columns]

final_query_time_2 =  (
    "SELECT \r\n"
    "    GREATEST(\r\n"
    "        " + ',\r\n        '.join(count_parts) + "\r\n"
    "    ) AS count_line\r\n"
    "   FROM \r\n"
    f"    {table_name}\r\n"
    "   WHERE \r\n"
    "    " + ' OR '.join([f"'{column}' IS NOT NULL" for column in columns]) + ";"
)

print(final_query_time_2)

conn.close()



SELECT 
    GREATEST(
        COUNT('atm_acquiring_country'),
        COUNT('atm_acquiring_icc'),
        COUNT('atm_branch'),
        COUNT('atm_card_entry_mode'),
        COUNT('atm_cardless'),
        COUNT('atm_cash_in'),
        COUNT('atm_cash_in_acc'),
        COUNT('atm_cash_in_pay'),
        COUNT('atm_city'),
        COUNT('atm_class'),
        COUNT('atm_continent'),
        COUNT('atm_country'),
        COUNT('atm_cvv_check'),
        COUNT('atm_exchange'),
        COUNT('atm_forwarding_icc'),
        COUNT('atm_is_fi_owner'),
        COUNT('atm_merchant_id'),
        COUNT('atm_merchant_name'),
        COUNT('atm_owner_fi_name'),
        COUNT('atm_pin_check'),
        COUNT('atm_pin_entry_mode'),
        COUNT('atm_receiving_icc'),
        COUNT('atm_region'),
        COUNT('atm_retailer_name'),
        COUNT('atm_usr_num_hit'),
        COUNT('atm_virt'),
        COUNT('atm_zip'),
        COUNT('branch_branch'),
        COUNT('branch_host'),
        COUNT('branch_office')

In [8]:
Time_2 = (
"table_name: Time_2",
"main_dttm_col: null",
"description: null",
"default_endpoint: null",
"offset: 0",
"cache_timeout: null",
"catalog: postgres",
"schema: public",
f"sql: {repr(final_query_time_2)} ",
"params: null",
"template_params: null",
"filter_select_enabled: true",
"fetch_values_predicate: null",
"extra: null",
"normalize_columns: false",
"always_filter_main_dttm: false",
"uuid: fd91a537-b418-4be5-92ca-c62d01b85d63",
"metrics:",
"- metric_name: count",
"  verbose_name: COUNT(*)",
"  metric_type: count",
"  expression: COUNT(*)",
"  description: null",
"  d3format: null",
"  currency: null",
"  extra: null",
"  warning_text: null",
"columns:",
"- column_name: count_line",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: LONGINTEGER",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"version: 1.0.0",
"database_uuid: 2f651e35-38e7-4b9b-a47e-87996c769118"
)

Time_3

In [9]:
text_columns = df.select_dtypes(include=['object']).columns.tolist()

query_parts = []
for column in text_columns:
    query_part = (
        f"  SELECT\r\n"
        f"    '{column}' AS column_name,\r\n"
        f"    COUNT(DISTINCT {column}) AS unique_count\r\n"
        f"  FROM {table_name}"
    )
    query_parts.append(query_part)

final_query_time_3 = "\r\n  UNION ALL\r\n".join(query_parts)

print(final_query_time_3)

  SELECT
    'atm_acquiring_country' AS column_name,
    COUNT(DISTINCT atm_acquiring_country) AS unique_count
  FROM anti_frod
  UNION ALL
  SELECT
    'atm_acquiring_icc' AS column_name,
    COUNT(DISTINCT atm_acquiring_icc) AS unique_count
  FROM anti_frod
  UNION ALL
  SELECT
    'atm_branch' AS column_name,
    COUNT(DISTINCT atm_branch) AS unique_count
  FROM anti_frod
  UNION ALL
  SELECT
    'atm_card_entry_mode' AS column_name,
    COUNT(DISTINCT atm_card_entry_mode) AS unique_count
  FROM anti_frod
  UNION ALL
  SELECT
    'atm_city' AS column_name,
    COUNT(DISTINCT atm_city) AS unique_count
  FROM anti_frod
  UNION ALL
  SELECT
    'atm_class' AS column_name,
    COUNT(DISTINCT atm_class) AS unique_count
  FROM anti_frod
  UNION ALL
  SELECT
    'atm_continent' AS column_name,
    COUNT(DISTINCT atm_continent) AS unique_count
  FROM anti_frod
  UNION ALL
  SELECT
    'atm_country' AS column_name,
    COUNT(DISTINCT atm_country) AS unique_count
  FROM anti_frod
  UNION ALL


In [10]:
Time_3 = (
"table_name: Time_3",
"main_dttm_col: null",
"description: null",
"default_endpoint: null",
"offset: 0",
"cache_timeout: null",
"catalog: postgres",
"schema: public",
f"sql: {(final_query_time_3)} ",
"params: null",
"template_params: null",
"filter_select_enabled: true",
"fetch_values_predicate: null",
"extra: null",
"normalize_columns: false",
"always_filter_main_dttm: false",
"uuid: 094fc1fa-42e9-427d-a9a1-0fa661fb5fa8",
"metrics:",
"- metric_name: count",
"  verbose_name: COUNT(*)",
"  metric_type: count",
"  expression: COUNT(*)",
"  description: null",
"  d3format: null",
"  currency: null",
"  extra: null",
"  warning_text: null",
"columns:",
"- column_name: unique_count",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: LONGINTEGER",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"- column_name: column_name",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: STRING",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"version: 1.0.0",
"database_uuid: 2f651e35-38e7-4b9b-a47e-87996c769118"
)

Time_4

In [11]:
columns =  df.columns.tolist()


column_list = "', '".join(columns)
final_query_time_4=(
    "SELECT\r\n"
    "   column_name AS \"Column_Name\",\r\n"
    "   data_type AS \"Data_Type\"\r\n"
    "   FROM\r\n"
    "       information_schema.columns\r\n"
    f"   WHERE table_name = '{table_name}'\r\n"
    f"   AND column_name IN ('{column_list}')"
)
final_query_time_4

'SELECT\r\n   column_name AS "Column_Name",\r\n   data_type AS "Data_Type"\r\n   FROM\r\n       information_schema.columns\r\n   WHERE table_name = \'anti_frod\'\r\n   AND column_name IN (\'atm_acquiring_country\', \'atm_acquiring_icc\', \'atm_branch\', \'atm_card_entry_mode\', \'atm_cardless\', \'atm_cash_in\', \'atm_cash_in_acc\', \'atm_cash_in_pay\', \'atm_city\', \'atm_class\', \'atm_continent\', \'atm_country\', \'atm_cvv_check\', \'atm_exchange\', \'atm_forwarding_icc\', \'atm_is_fi_owner\', \'atm_merchant_id\', \'atm_merchant_name\', \'atm_owner_fi_name\', \'atm_pin_check\', \'atm_pin_entry_mode\', \'atm_receiving_icc\', \'atm_region\', \'atm_retailer_name\', \'atm_usr_num_hit\', \'atm_virt\', \'atm_zip\', \'branch_branch\', \'branch_host\', \'branch_office\', \'branch_operator_id\', \'branch_tercode\', \'card_balance\', \'card_balance_currency\', \'card_bin\', \'card_category\', \'card_contract_currency\', \'card_day_limit\', \'card_day_limit_currency\', \'card_days_since_issue

In [12]:
Time_4 = (
"table_name: Time_4",
"main_dttm_col: null",
"description: null",
"default_endpoint: null",
"offset: 0",
"cache_timeout: null",
"catalog: postgres",
"schema: public",
f"sql: {(final_query_time_4)} ",
"params: null",
"template_params: null",
"filter_select_enabled: true",
"fetch_values_predicate: null",
"extra: null",
"normalize_columns: false",
"always_filter_main_dttm: false",
"uuid: 763ef388-5be4-46bb-b7e7-f0fb3254eeff",
"metrics:",
"- metric_name: count",
"  verbose_name: COUNT(*)",
"  metric_type: count",
"  expression: COUNT(*)",
"  description: null",
"  d3format: null",
"  currency: null",
"  extra: null",
"  warning_text: null",
"columns:",
"- column_name: Column_Name",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: STRING",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"- column_name: Data_Type",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: STRING",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"version: 1.0.0",
"database_uuid: 2f651e35-38e7-4b9b-a47e-87996c769118"
)

Time_5

In [13]:
columns =  df.columns.tolist()


column_list = "', '".join(columns)
final_query_time_5 = (
    "SELECT\r\n"
    "    COUNT(*) FILTER (WHERE data_type ILIKE 'character varying' OR data_type ILIKE 'text') AS text_column_count,\r\n"
    "    COUNT(*) FILTER (WHERE data_type ILIKE 'integer' OR data_type ILIKE 'numeric' OR data_type ILIKE 'double precision' OR data_type ILIKE 'real') AS numeric_column_count,\r\n"
    "    COUNT(*) FILTER (WHERE data_type ILIKE 'TIMESTAMP' OR data_type ILIKE 'time' OR data_type ILIKE 'date') AS time_column_count\r\n"
    "   FROM information_schema.columns\r\n"
    f"   WHERE table_name = '{table_name}'\r\n"
    f"   AND column_name IN ('{column_list}')"
)
final_query_time_5

"SELECT\r\n    COUNT(*) FILTER (WHERE data_type ILIKE 'character varying' OR data_type ILIKE 'text') AS text_column_count,\r\n    COUNT(*) FILTER (WHERE data_type ILIKE 'integer' OR data_type ILIKE 'numeric' OR data_type ILIKE 'double precision' OR data_type ILIKE 'real') AS numeric_column_count,\r\n    COUNT(*) FILTER (WHERE data_type ILIKE 'TIMESTAMP' OR data_type ILIKE 'time' OR data_type ILIKE 'date') AS time_column_count\r\n   FROM information_schema.columns\r\n   WHERE table_name = 'anti_frod'\r\n   AND column_name IN ('atm_acquiring_country', 'atm_acquiring_icc', 'atm_branch', 'atm_card_entry_mode', 'atm_cardless', 'atm_cash_in', 'atm_cash_in_acc', 'atm_cash_in_pay', 'atm_city', 'atm_class', 'atm_continent', 'atm_country', 'atm_cvv_check', 'atm_exchange', 'atm_forwarding_icc', 'atm_is_fi_owner', 'atm_merchant_id', 'atm_merchant_name', 'atm_owner_fi_name', 'atm_pin_check', 'atm_pin_entry_mode', 'atm_receiving_icc', 'atm_region', 'atm_retailer_name', 'atm_usr_num_hit', 'atm_virt',

In [14]:
Time_5 = (
"table_name: Time_5",
"main_dttm_col: null",
"description: null",
"default_endpoint: null",
"offset: 0",
"cache_timeout: null",
"catalog: postgres",
"schema: public",
f"sql: {repr(final_query_time_5)} ",
"params: null",
"template_params: null",
"filter_select_enabled: true",
"fetch_values_predicate: null",
"extra: null",
"normalize_columns: false",
"always_filter_main_dttm: false",
"uuid: d661cc21-5919-4136-aa9d-cec6f3886047",
"metrics:",
"- metric_name: count",
"  verbose_name: COUNT(*)",
"  metric_type: count",
"  expression: COUNT(*)",
"  description: null",
"  d3format: null",
"  currency: null",
"  extra: null",
"  warning_text: null",
"columns:",
"- column_name: numeric_column_count",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: LONGINTEGER",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"- column_name: time_column_count",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: LONGINTEGER",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"- column_name: text_column_count",
"  verbose_name: null",
"  is_dttm: false",
"  is_active: true",
"  type: LONGINTEGER",
"  advanced_data_type: null",
"  groupby: true",
"  filterable: true",
"  expression: null",
"  description: null",
"  python_date_format: null",
"  extra: null",
"version: 1.0.0",
"database_uuid: 2f651e35-38e7-4b9b-a47e-87996c769118"
)

Time_6

In [15]:
import io
from minio import Minio
from minio.error import S3Error

# Конфигурация MinIO
endpoint = "127.0.0.1:32020"  # Адрес MinIO сервера
access_key = "qxiYSHv8QP4QE0Ctk0fI"  # Ваш Access Key
secret_key = "34w2smESi4yNjxS9ga1ibJqhXGpdSerTsSnvkQHH"
bucket_name = "dashbord-paket"  # Название бакета

# Инициализируем клиента MinIO
client = Minio(
    endpoint,
    access_key=access_key,
    secret_key=secret_key,
    secure=False  # Установите True, если используется HTTPS
)

def upload_to_minio(bucket, file_path, content):
    """
    Загрузить содержимое файла в MinIO.

    :param bucket: Название бакета
    :param file_path: Путь к файлу внутри бакета
    :param content: Содержимое файла для записи (строка или bytes)
    """
    try:
        # Убедитесь, что бакет существует
        if not client.bucket_exists(bucket):
            client.make_bucket(bucket)

        # Преобразуем содержимое в поток
        if isinstance(content, str):
            content = content.encode('utf-8')  # Кодируем строку в bytes
        data_stream = io.BytesIO(content)  # Создаём поток

        # Загружаем данные в MinIO
        client.put_object(
            bucket,
            file_path,
            data=data_stream,
            length=len(content),
            content_type="text/yaml"
        )
        print(f"Файл '{file_path}' успешно загружен в бакет '{bucket}'.")
    except S3Error as err:
        print(f"Ошибка загрузки в MinIO: {err}")

# Список файлов для записи
time_files = {
    "dashboard_export_time/datasets/PostgreSQL/Time_5.yaml": '\n'.join(Time_5),
    "dashboard_export_time/datasets/PostgreSQL/Time_4.yaml": '\n'.join(Time_4),
    "dashboard_export_time/datasets/PostgreSQL/Time_3.yaml": '\n'.join(Time_3),
    "dashboard_export_time/datasets/PostgreSQL/Time_2.yaml": '\n'.join(Time_2),
    "dashboard_export_time/datasets/PostgreSQL/Time_1.yaml": '\n'.join(Time_1),
}

# Загрузка файлов в MinIO
for file_path, content in time_files.items():
    upload_to_minio(bucket_name, file_path, content.encode('utf-8'))


Файл 'dashboard_export_time/datasets/PostgreSQL/Time_5.yaml' успешно загружен в бакет 'dashbord-paket'.
Файл 'dashboard_export_time/datasets/PostgreSQL/Time_4.yaml' успешно загружен в бакет 'dashbord-paket'.
Файл 'dashboard_export_time/datasets/PostgreSQL/Time_3.yaml' успешно загружен в бакет 'dashbord-paket'.
Файл 'dashboard_export_time/datasets/PostgreSQL/Time_2.yaml' успешно загружен в бакет 'dashbord-paket'.
Файл 'dashboard_export_time/datasets/PostgreSQL/Time_1.yaml' успешно загружен в бакет 'dashbord-paket'.


In [21]:
import io
import zipfile
from minio import Minio
from minio.error import S3Error

# Конфигурация MinIO
endpoint = "127.0.0.1:32020"  # Адрес MinIO сервера
access_key = "qxiYSHv8QP4QE0Ctk0fI"  # Ваш Access Key
secret_key = "34w2smESi4yNjxS9ga1ibJqhXGpdSerTsSnvkQHH"  # Ваш Secret Key
bucket_name = "dashbord-paket"  # Название бакета
folder_prefix = "innstd_time_no_time_6/"  # Префикс папки
output_zip_name = "innstd_time_no_time_6.zip"  # Название архива

# Инициализация клиента MinIO
client = Minio(
    endpoint,
    access_key=access_key,
    secret_key=secret_key,
    secure=False  # Установите True, если используется HTTPS
)

def zip_folder_from_minio(bucket, folder_prefix, output_zip_name):
    """
    Сжать всю папку в архив и сохранить её в MinIO.
    
    :param bucket: Название бакета
    :param folder_prefix: Префикс папки в бакете (например, 'dashboard_export_time/')
    :param output_zip_name: Название ZIP файла для сохранения в MinIO
    """
    try:
        # Список объектов в указанной "папке"
        objects = client.list_objects(bucket, prefix=folder_prefix, recursive=True)
        zip_stream = io.BytesIO()

        # Создаём архив ZIP в памяти
        with zipfile.ZipFile(zip_stream, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for obj in objects:
                # Скачиваем объект в память
                obj_data = client.get_object(bucket, obj.object_name)
                file_data = obj_data.read()

                # Добавляем объект в ZIP архив с сохранением структуры
                relative_path = obj.object_name  # Сохраняем полный путь в архиве
                zipf.writestr(relative_path, file_data)
                obj_data.close()

        # Перемещаем указатель в начало потока
        zip_stream.seek(0)

        # Сохраняем ZIP архив обратно в MinIO
        client.put_object(
            bucket,
            output_zip_name,
            data=zip_stream,
            length=len(zip_stream.getvalue()),
            content_type="application/zip"
        )
        print(f"Архив '{output_zip_name}' успешно создан в бакете '{bucket}'.")
    except S3Error as err:
        print(f"Ошибка работы с MinIO: {err}")

# Пример вызова функции
zip_folder_from_minio(bucket_name, folder_prefix, output_zip_name)


Архив 'innstd_time_no_time_6.zip' успешно создан в бакете 'dashbord-paket'.


In [23]:
import requests
import io
from minio import Minio

# Конфигурация Superset
SUPERSET_PROTOCOL = "http"
SUPERSET_HOST = "localhost:8088"
SUPERSET_USERNAME = "admin"
SUPERSET_PASSWORD = "admin"

# Конфигурация MinIO
class Importer:
    def __init__(self):
        self.session = requests.Session()
        self.get_superset_access_token()
        self.get_csrf_token()

    def get_superset_access_token(self):
        # URL для получения токена
        endpoint = "/api/v1/security/login"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Данные для аутентификации
        credentials = {
            "username": SUPERSET_USERNAME,
            "password": SUPERSET_PASSWORD,
            "provider": "db",
            "refresh": True
        }

        # Запрос на получение токена
        response = self.session.post(url, json=credentials)
        
        if response.status_code == 200:
            access_token = response.json().get('access_token')
            self.session.headers.update({
                'Authorization': f'Bearer {access_token}'
            })
        else:
            print("Ошибка при получении токена:", response.status_code, response.text)
            raise Exception(f"Ошибка получения токена: {response.status_code}")

    def get_csrf_token(self):
        # URL для получения CSRF-токена
        endpoint = "/api/v1/security/csrf_token/"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Запрос на получение CSRF-токена
        response = self.session.get(url)
        if response.status_code == 200:
            csrf_token = response.json().get("result")
            self.session.headers.update({
                'X-CSRFToken': csrf_token
            })
        else:
            print("Ошибка при получении CSRF токена:", response.status_code, response.text)
            raise Exception(f"Ошибка получения CSRF токена: {response.status_code}")

    def import_dashboard(self, zip_data_stream):
        # URL для импорта дашборда
        endpoint = "/api/v1/dashboard/import/"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Формируем данные для отправки
        files = {'formData': ("innstd_time_no_time_6.zip", zip_data_stream, 'application/zip')}
        payload = {
            'overwrite': 'true'
        }

        # Отправляем запрос на импорт дашборда
        response = self.session.post(url, files=files, data=payload)

        if response.status_code == 200:
            print("Дашборд успешно импортирован.")
        else:
            print("Ошибка при импорте дашборда:", response.status_code, response.text)


def get_file_from_minio(bucket, file_name):
    """
    Загружает файл из MinIO и возвращает его как поток байтов.
    
    :param bucket: Название бакета
    :param file_name: Имя файла в бакете
    :return: Поток данных (BytesIO)
    """
    try:
        # Загружаем файл в память
        response = client.get_object(bucket, file_name)
        file_data = io.BytesIO(response.read())
        response.close()
        response.release_conn()
        print(f"Файл '{file_name}' успешно загружен из MinIO.")
        return file_data
    except S3Error as err:
        print(f"Ошибка при работе с MinIO: {err}")
        raise


if __name__ == "__main__":
    try:
        # Получаем файл из MinIO
        zip_data_stream = get_file_from_minio(bucket_name, output_zip_name)

        # Импортируем дашборд
        importer = Importer()
        importer.import_dashboard(zip_data_stream)
    except Exception as e:
        print(f"Ошибка выполнения: {e}")


Файл 'innstd_time_no_time_6.zip' успешно загружен из MinIO.
Дашборд успешно импортирован.
